In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import ElasticNet
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


## Loading the dataset

In [41]:
df = pd.read_csv('pd_EEG_features.csv')
df.head()

,id,gender,PPE,EEG,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [42]:
X = df.drop(columns=['id','class']) 
y = df['class']  

In [43]:
count0=0
count1=0
for i in y:
    if i==0:
        count0+=1
    else:
        count1+=1

print(count0,count1)

192 564


## Balancing the dataset

In [44]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
df_resampled = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled, columns=['class'])], axis=1)

In [45]:
df_resampled

,gender,PPE,EEG,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,0.852470,0.718260,0.572270,240,239,0.008064,0.000087,0.002180,0.000018,...,1.562000,2.644500,3.868600,4.210500,5.122100,4.462500,2.620200,3.000400,18.940500,1
1,1,0.766860,0.694810,0.539660,234,233,0.008258,0.000073,0.001950,0.000016,...,1.558900,3.610700,23.515500,14.196200,11.026100,9.508200,6.524500,6.343100,45.178000,1
2,1,0.850830,0.676040,0.589820,232,231,0.008340,0.000060,0.001760,0.000015,...,1.564300,2.330800,9.495900,10.745800,11.017700,4.806600,2.919900,3.149500,4.766600,1
3,0,0.411210,0.796720,0.592570,178,177,0.010858,0.000183,0.004190,0.000046,...,3.780500,3.566400,5.255800,14.040300,4.223500,4.685700,4.846000,6.265000,4.060300,1
4,0,0.327900,0.797820,0.530280,236,235,0.008162,0.002669,0.005350,0.000044,...,6.172700,5.841600,6.080500,5.762100,7.781700,11.689100,8.210300,5.055900,6.116400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,0,0.846790,0.672417,0.462422,504,503,0.003835,0.000061,0.000767,0.000003,...,8.647863,4.877069,4.029061,3.721892,3.927724,4.420524,4.450813,4.594596,4.238312,0
1124,0,0.830738,0.696754,0.353928,440,439,0.004385,0.000032,0.000781,0.000003,...,54.538588,52.496380,36.303025,22.949557,15.209481,12.072220,14.473528,14.720977,29.266257,0
1125,0,0.822565,0.563505,0.413189,493,492,0.003914,0.000045,0.000547,0.000002,...,86.697327,58.214193,47.425700,39.920400,31.880980,26.469694,19.636002,10.795145,3.076334,0
1126,0,0.787599,0.609335,0.207256,594,593,0.003245,0.000033,0.000373,0.000001,...,188.246003,146.915555,79.392584,38.204836,31.944149,30.986111,13.733797,9.864585,13.058255,0


In [46]:
X = df_resampled.drop(columns=['class']) 
y = df_resampled['class']  

In [47]:
count0=0
count1=0
for i in y:
    if i==0:
        count0+=1
    else:
        count1+=1

print(count0,count1)

564 564


In [48]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### XGBoost

In [49]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

In [50]:
xg_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - xg_accuracy
y_pred_proba = xgb.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",xg_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9424778761061947
Precision: 0.9642857142857143
Recall: 0.9230769230769231
F1 Score: 0.9432314410480349
Error Rate: 0.05752212389380529
Binary Cross-Entropy Loss: 0.1126669994341628
Confusion Matrix:
[[105   4]
 [  9 108]]


### AdaBoost Classifier

In [51]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [52]:
AB_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - AB_accuracy
y_pred_proba = ada.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",AB_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9513274336283186
Precision: 0.9649122807017544
Recall: 0.9401709401709402
F1 Score: 0.9523809523809523
Error Rate: 0.04867256637168138
Binary Cross-Entropy Loss: 0.635890992699611
Confusion Matrix:
[[105   4]
 [  7 110]]


### CatBoost

In [53]:
catboost_classifier = CatBoostClassifier(n_estimators=100, random_state=42)
catboost_classifier.fit(X_train, y_train)
y_pred = catboost_classifier.predict(X_test)

Learning rate set to 0.081434
0:	learn: 0.6399365	total: 54.6ms	remaining: 5.41s
1:	learn: 0.5980812	total: 108ms	remaining: 5.3s
2:	learn: 0.5565662	total: 169ms	remaining: 5.47s
3:	learn: 0.5246087	total: 229ms	remaining: 5.48s
4:	learn: 0.4959610	total: 288ms	remaining: 5.47s
5:	learn: 0.4676046	total: 342ms	remaining: 5.36s
6:	learn: 0.4360529	total: 404ms	remaining: 5.37s
7:	learn: 0.4145788	total: 459ms	remaining: 5.27s
8:	learn: 0.3943257	total: 515ms	remaining: 5.21s
9:	learn: 0.3776635	total: 570ms	remaining: 5.13s
10:	learn: 0.3634121	total: 639ms	remaining: 5.17s
11:	learn: 0.3507654	total: 694ms	remaining: 5.09s
12:	learn: 0.3354181	total: 746ms	remaining: 5s
13:	learn: 0.3226549	total: 798ms	remaining: 4.9s
14:	learn: 0.3108936	total: 852ms	remaining: 4.83s
15:	learn: 0.3023147	total: 907ms	remaining: 4.76s
16:	learn: 0.2917577	total: 962ms	remaining: 4.69s
17:	learn: 0.2820982	total: 1.01s	remaining: 4.62s
18:	learn: 0.2702501	total: 1.07s	remaining: 4.55s
19:	learn: 0.26

In [54]:
catboost_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - catboost_accuracy
y_pred_proba = catboost_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",catboost_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9690265486725663
Precision: 0.9824561403508771
Recall: 0.9572649572649573
F1 Score: 0.9696969696969697
Error Rate: 0.030973451327433676
Binary Cross-Entropy Loss: 0.1265617108437408
Confusion Matrix:
[[107   2]
 [  5 112]]


### GBM

In [55]:
gbm = GradientBoostingClassifier(
    n_estimators=100,      
    learning_rate=0.05,
    max_depth=5,          
    max_features=0.9,
    random_state=42
)
gbm.fit(X_train, y_train)

y_pred_prob = gbm.predict_proba(X_test)[:, 1]
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred_prob]


In [56]:
gbm_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - gbm_accuracy
y_pred_proba = gbm.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",gbm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9424778761061947
Precision: 0.9824561403508771
Recall: 0.9572649572649573
F1 Score: 0.9696969696969697
Error Rate: 0.05752212389380529
Binary Cross-Entropy Loss: 0.1260870345126343
Confusion Matrix:
[[107   2]
 [  5 112]]


### LightGBM

In [57]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
lgbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval)
y_pred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]


[LightGBM] [Info] Number of positive: 447, number of negative: 455
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019593 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191489
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 753
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495565 -> initscore=-0.017739
[LightGBM] [Info] Start training from score -0.017739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [58]:
lgbm_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
error_rate = 1 - lgbm_accuracy
y_pred_proba = 1 / (1 + np.exp(-y_pred))
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print("Accuracy:",lgbm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9734513274336283
Precision: 0.9911504424778761
Recall: 0.9572649572649573
F1 Score: 0.9739130434782609
Error Rate: 0.026548672566371723
Binary Cross-Entropy Loss: 0.5305929448832605
Confusion Matrix:
[[108   1]
 [  5 112]]


### Ridge Classifier

In [59]:
ridge_classifier = RidgeClassifier(
    alpha=1.0,  
    random_state=42
)
ridge_classifier.fit(X_train, y_train)
y_pred = ridge_classifier.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)

C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=4.5274e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [60]:
ridge_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - ridge_accuracy
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",ridge_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8982300884955752
Precision: 0.97
Recall: 0.8290598290598291
F1 Score: 0.8940092165898618
Error Rate: 0.10176991150442483
Confusion Matrix:
[[106   3]
 [ 20  97]]


### ElasticNet 

In [61]:
elastic_net_classifier = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
elastic_net_classifier.fit(X_train, y_train)
y_pred = elastic_net_classifier.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.936e+01, tolerance: 2.255e-02
  model = cd_fast.enet_coordinate_descent(


In [62]:
elastic_net_accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
error_rate = 1 - elastic_net_accuracy
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print("Accuracy:",elastic_net_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8761061946902655
Precision: 0.908256880733945
Recall: 0.8461538461538461
F1 Score: 0.8761061946902655
Error Rate: 0.12389380530973448
Confusion Matrix:
[[99 10]
 [18 99]]


### RandomForest

In [63]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)


In [64]:
rf_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - rf_accuracy
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",rf_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9380530973451328
Precision: 0.963963963963964
Recall: 0.9145299145299145
F1 Score: 0.9385964912280702
Error Rate: 0.06194690265486724
Binary Cross-Entropy Loss: 0.23266725444465847
Confusion Matrix:
[[105   4]
 [ 10 107]]


### ExtraTrees

In [65]:
et_classifier = ExtraTreesClassifier(n_estimators=100, random_state=42)
et_classifier.fit(X_train, y_train)
y_pred = et_classifier.predict(X_test)

In [66]:
et_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - et_accuracy
y_pred_proba = et_classifier.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",et_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9469026548672567
Precision: 0.981651376146789
Recall: 0.9145299145299145
F1 Score: 0.9469026548672567
Error Rate: 0.053097345132743334
Binary Cross-Entropy Loss: 0.18748306756059469
Confusion Matrix:
[[107   2]
 [ 10 107]]


### GMM

In [67]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
gmm = GaussianMixture(n_components=2, random_state=42)
gmm.fit(X_scaled)
cluster_labels = gmm.predict(X_scaled)

def map_cluster_labels(true_labels, cluster_labels):
    new_labels = np.zeros_like(cluster_labels)
    for i in range(np.max(cluster_labels) + 1):
        mask = (cluster_labels == i)
        new_labels[mask] = mode(true_labels[mask])[0]
    return new_labels

true_labels = y  
mapped_labels = map_cluster_labels(true_labels, cluster_labels)


In [68]:
gmm_accuracy = accuracy_score(true_labels, mapped_labels)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
error_rate = 1 - gmm_accuracy
y_pred_proba = gmm.predict_proba(X_test)[:, 1]
loss = log_loss(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:",gmm_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error Rate:", error_rate)
print("Binary Cross-Entropy Loss:", loss)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.7313829787234043
Precision: 0.981651376146789
Recall: 0.9145299145299145
F1 Score: 0.9469026548672567
Error Rate: 0.2686170212765957
Binary Cross-Entropy Loss: 18.659767462507553
Confusion Matrix:
[[107   2]
 [ 10 107]]


C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(


### Accuracy on UNBALANCED data

In [69]:
print(f'XGBoost Accuracy: {xg_accuracy}')
print(f'AdaBoost Accuracy: {AB_accuracy}')
print(f"CatBoost Accuracy:{catboost_accuracy}")
print(f'GBM Accuracy: {gbm_accuracy}')
print(f'LightGBM Accuracy: {lgbm_accuracy}')
print(f'Ridge Accuracy: {ridge_accuracy}')
print(f'ElasticNet Accuracy: {elastic_net_accuracy}')
print(f'RandomForest Accuracy: {rf_accuracy}')
print(f'ExtraTrees Accuracy: {et_accuracy}')
print(f'GMM Accuracy: {gmm_accuracy}')

XGBoost Accuracy: 0.9424778761061947
AdaBoost Accuracy: 0.9513274336283186
CatBoost Accuracy:0.9690265486725663
GBM Accuracy: 0.9424778761061947
LightGBM Accuracy: 0.9734513274336283
Ridge Accuracy: 0.8982300884955752
ElasticNet Accuracy: 0.8761061946902655
RandomForest Accuracy: 0.9380530973451328
ExtraTrees Accuracy: 0.9469026548672567
GMM Accuracy: 0.7313829787234043
